In [89]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df_firms = pd.read_csv(r"W:\019_Glassdoor\1 Data\1 Glassdoor Links\0816_LinksNamesReviews_bestmatch.csv", sep=";", decimal=",", encoding='unicode_escape')

## sub df
df_firms = df_firms[["isin", "company_name", "best_match"]]

## filter out firm names with less than 4 characters. eikon search goes crazy with 3 character firms (eg "UST")

df_firms = df_firms[df_firms["best_match"].str.len() > 3]
df_firms.head()

## convert best_match row to list, which can be used in get_data function
firms_list = df_firms["best_match"].tolist()
firms_list = firms_list[:10]

,isin,company_name,best_match
0,US9668371068,Whole Foods Market,Whole Foods Market
1,US87652V1098,TaskUs,TaskUs
2,US9285634021,VMware,VMware
3,US1567821046,Cerner,Cerner
4,US98421M1062,Xerox,Xerox


In [2]:
## https://developers.refinitiv.com/article/find-your-right-companies-screener-eikon-data-apispython
import eikon as ek
ek.set_app_key('341ab68907a740f38e7b07084d4055e3982b5bdf')

In [86]:
screener_func = f'SCREEN(U(IN(Equity(active or inactive, primary))), IN(TR.ExchangeCountryCode,US) OR IN(TR.ExchangeCountryCode,CA), Contains(TR.CommonName, "Whole Foods Market") )'
instruments = ["TR.ISIN", "TR.CompanyName"]

df, err = ek.get_data(screener_func, instruments) 
df

,Instrument,ISIN,Company Name
0,WFM.OQ^H17,US9668371068,Whole Foods Market Inc


In [91]:
firms_list_screenconverted = "".join([f"Contains(TR.CommonName,'{firm}') OR " if index != len(firms_list)-1 else f"Contains(TR.CommonName,'{firm}')" for index, firm in enumerate(firms_list)])
firms_list_screenconverted

screener_func = f'SCREEN(U(IN(Equity(active or inactive,primary))), IN(TR.ExchangeCountryCode,US) OR IN(TR.ExchangeCountryCode,CA), {firms_list_screenconverted} )'
instruments = ["TR.ISIN", "TR.CompanyName"]

df, err = ek.get_data(screener_func, instruments) 
df

"Contains(TR.CommonName,'Whole Foods Market') OR Contains(TR.CommonName,'TaskUs') OR Contains(TR.CommonName,'VMware') OR Contains(TR.CommonName,'Cerner') OR Contains(TR.CommonName,'Xerox') OR Contains(TR.CommonName,'Tim Hortons') OR Contains(TR.CommonName,'Atos Syntel') OR Contains(TR.CommonName,'Raytheon Technologies') OR Contains(TR.CommonName,'Tesla') OR Contains(TR.CommonName,'Convergys')"

,Instrument,ISIN,Company Name
0,WFM.OQ^H17,US9668371068,Whole Foods Market Inc
1,TASK.OQ,US87652V1098,Taskus Inc
2,VMW.N,US9285634021,VMware Inc
3,CERN.OQ^F22,US1567821046,Cerner Corp
4,XRX.OQ,US98421M1062,Xerox Holdings Corp
5,XER_pa.TO^C06,TCP989431060,Xerox Capital LLC
6,GISX.OQ^E07,US37934A1007,Xerox Business Solutions Inc
7,THI.TO^L14,CA88706M1032,Tim Hortons Inc (Pre-Merger)
8,RTX.N,US75513E1010,Raytheon Technologies Corp
9,TXLZF.PK,CA8816011081,Tesla Exploration Ltd
